<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_5/01_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 学術分野の分類（複数）
## jst_categorize_multi.ipynb
kakenhi_fine_tuning.ipynbでファインチューニングにより作成したモデルを用いて、新たな複数の学術文書（JSTやAMEDを含めた科研費に限らない）を11の大区分に分類するプログラム。  

### ライブラリのインストール
SageMakerでは初回だけで良い（Google colaboratoryでは毎回）  
ライブラリTransformers、およびnlpをインストールします。  

In [ ]:
# pipのアップデート（たまに走らせても良いかも）
!pip list
!python -m pip install --upgrade pip

In [ ]:
# ライブラリのインストール
!pip install torch
!pip install transformers
!pip install nlp
!pip install datasets
!pip install fugashi
!pip install ipadic

### Google ドライブとの連携  
以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [ ]:
# SageMakerでは不要
# from google.colab import drive
# drive.mount("/content/drive/")

## 複数の学術文書を分類
学習済み（ファインチューニング済み）のモデルを読み込み、複数の学術文書（実際には科研費概要テキスト）を分類する

### モデルの読み込み
保存済みのモデルを読み込みます。

In [ ]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer
print("Start")
#data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # SageMaker

loaded_model = BertForSequenceClassification.from_pretrained(data_path) 
loaded_model.cuda() # GPU対応
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(data_path)

print("Finish")

### 分類精度検証用の科研費データ読み込み

In [12]:
import pandas as pd

print("Start")

# open_original_csv = "RUC共同研究_概要_NoJSPS.csv" #
# open_original_csv = "RUC共同研究_概要_NoDescription.csv" # 
# open_original_csv = "再々依頼_概要なし.csv" # 
open_original_csv = "再々依頼_概要あり.csv" # 
data_path = "../data/"

# csvファイルを開く
raw_data2 = pd.read_csv(data_path + open_original_csv) # dtype="object"必要？

# 読み込んだデータをチェック
# raw_data.info()


# 今後必要な行だけを取り出し、リネーム
kadaiDai2 = raw_data2[["ID", "FA", "description"]]
# kadaiDai2 = raw_data2[["ID", "FA", "Title"]] # only for NoDescription
kadaiDai2['tabDai'] = 0
kadaiDai2.columns = ["ID", "FA", "Abst", "tabDai"]

# kadaiDai2 = kadaiDai2[kadaiDai2['FA'] != 'JSPS']

print("Finish")
kadaiDai2

Start


/tmp/ipykernel_85/1531393593.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kadaiDai2['tabDai'] = 0


,ID,FA,Abst,tabDai
0,1009959,環境省,"・211At,223Ra および 225Ac と壊変核種について、精製・標識実験、細胞実験、...",0
1,918584,AMED,・ヒト肝細胞キメラ TK-NOG マウスに C 型肝炎患者血清を投与することにより、ウイルス...,0
2,886054,厚労省,・医療機関外死亡に対する死後画像診断を安全に実施するための基準を作成する。・死後画像診断につ...,0
3,955716,厚労省,・感染症発生動向調査の評価と改善法の提案：疫学的・統計学的な観点からの評価とともに、ステーク...,0
4,918227,AMED,・局所進行頭頸部扁平上皮癌術後再発High-Risk患者に対するweekly CDDP+RT...,0
...,...,...,...,...
2697,918254,AMED,膵癌に対する高い抗腫瘍効果が認められた人工核酸YB-1阻害アンチセンスを臨床応用するために、...,0
2698,917996,AMED,膵癌は、我が国で約3万人が年間死亡し、部位別癌死亡数の第5位を占める重要な悪性腫瘍である。膵...,0
2699,918255,AMED,膵癌は浸潤・転移能が非常に高く、転移抑制剤は膵癌の進行を抑制する戦略として期待される。われわ...,0
2700,147197,厚労省,臍帯血は血液疾患のみならず、組織幹細胞ソースとして代謝性疾患等にも応用され、さらに近年では脳...,0


In [13]:
print("Start")

# 課題番号の重複を確認。課題番号でソートする。
kadaiDai2["ID"].duplicated().any()
# kadaiDai2 = kadaiDai2.set_index("ID") # IDをインデックスに設定するコード
kadaiDai2 = kadaiDai2.sort_values("ID")

# Abstが空欄の課題を削除
print("オリジナルの課題数： %5d" % len(kadaiDai2))
print("概要が空白の課題数： %5d" % len(kadaiDai2[kadaiDai2["Abst"].isna()]))
kadaiDai2 = kadaiDai2.dropna(subset=["Abst"])
print("空白を除いた課題数： %5d" % len(kadaiDai2))

# Abst中の改行コードを削除
kadaiDai2 = kadaiDai2.replace('\r', '', regex=True)
kadaiDai2 = kadaiDai2.replace('\n', '', regex=True)
kadaiDai2 = kadaiDai2.replace('　', '', regex=True) # 全角スペース

# Abstが英語のみの課題を削除
num_jpen = len(kadaiDai2)
kadaiDai2 = kadaiDai2[kadaiDai2["Abst"].str.contains(r'[ぁ-んァ-ン]')]
num_jp   = len(kadaiDai2)
print("日本語＋英語： %5d" % num_jpen)
print("英語　　　　： %5d" % (num_jpen - num_jp))
print("日本語　　　： %5d" % num_jp)

print("Finish")

Start
オリジナルの課題数：  2702
概要が空白の課題数：     0
空白を除いた課題数：  2702
日本語＋英語：  2702
英語　　　　：     0
日本語　　　：  2702
Finish


### 分類精度を複数ファイルで確認
３時間程度かかるかも  
sagemakerでは40分程度

In [14]:
import glob  # ファイルの取得に使用
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

results_binary = 'results'
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]


max_length = 512

num_data = len(kadaiDai2)
print("分類する課題数： %d" % num_data)
# num_data = 100
num_category = len(Daikubun)
# results = torch.zeros(num_data, num_category) # テンソルで変数を用意
results = np.zeros((num_data, num_category)) # メモリーが足りないと言われるのでテンソルではなくnumpy arrayにしてみた

for m in tqdm(range(num_data)):
  words = loaded_tokenizer.tokenize(kadaiDai2["Abst"][m])
  word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
  word_tensor = torch.tensor([word_ids[:max_length]])  # テンソルに変換
  
  # y = loaded_model(word_tensor) # GPU未対応時の予測
  y = loaded_model(word_tensor.cuda())  # GPU対応時の予測

  # results[m,:] = y[0]
  results[m,:] = y[0].cpu().detach().numpy()  # テンソルをCPUにコピーしてからNumPy配列に変換
  # results[m,:] = y[0].detach().numpy() # テンソルをnumpy arrayに変換

# 変数をとりあえずバイナリで保存
np.save(data_path+results_binary, results) # 計算結果をとりあえずバイナリで保存

print(results.shape)
results

分類する課題数： 2702


100%|██████████| 2702/2702 [01:34<00:00, 28.68it/s]

(2702, 11)


array([[ 1.93207276, -1.15442955, -0.10450167, ...,  4.89918423,
        -0.84664804,  0.89417267],
       [-1.07828379, -3.08552122, -3.09622383, ...,  5.60167027,
        -2.12422323, -0.8098672 ],
       [ 0.94985783, -2.25845003, -1.42935514, ...,  6.35273838,
        -0.11959677, -2.22099066],
       ...,
       [-0.24932507, -2.79833794, -3.0353148 , ...,  6.83905315,
        -1.79406428, -1.44619238],
       [-0.71411115, -2.66774964, -2.41473103, ...,  6.94206619,
        -1.89559722, -1.50186241],
       [-0.06287571, -2.43212104, -2.33239222, ...,  6.98633909,
        -1.87043726, -1.43922555]])

### ここで一度保存
時間がかかる処理のあとなので、この段階で生成できたデータをファイルに保存

In [15]:
# コードが不十分
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]

results = np.load(data_path+results_binary+".npy")

# 変数をとりあえずバイナリで保存
#np.save(data_path+results_binary, results) # 計算結果をとりあえずバイナリで保存
np.savez(data_path+"categolization_results", results, kadaiDai2, Daikubun)

## 分類結果を保存・表示

In [18]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

show_num = 0

results_binary = "results"
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]

# num_category = len(Daikubun)

results_2 = np.load(data_path+results_binary+".npy")



#results_3 = np.load(data_path+"categolization_results.npz", allow_pickle=True)
#results_2 = results_3['arr_0']
#kadaiDai2 = results_3['arr_1']
#Daikubun  = results_3['arr_2']

print("ロードした推定確率データのサイズ %d %d" % results_2.shape)

num_category = len(Daikubun)

results_2 = torch.tensor(results_2) # Softmax関数を使うためにテンソルに変換
m = torch.nn.Softmax(dim=1) # Softmax関数で確率に変換
results_2 = m(results_2)
results_2 = results_2.numpy() # numpy arrayに戻す
df = pd.DataFrame(results_2, columns = Daikubun) # DataFrameに変換

# 保存
df.to_csv('output.csv', encoding='shift-jis')
print('Saved\n')

# 画面上に表示する目的
df = (df * 100).astype(int)
df

ロードした推定確率データのサイズ 2702 11
Saved



,A,B,C,D,E,F,G,H,I,J,K
0,4,0,0,1,0,0,0,0,90,0,1
1,0,0,0,0,0,3,0,7,88,0,0
2,0,0,0,0,0,0,0,0,97,0,0
3,1,0,0,1,0,0,0,1,94,0,0
4,0,0,0,0,0,0,0,0,99,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2697,0,0,0,0,0,0,0,2,96,0,0
2698,0,0,0,5,0,0,0,1,92,0,0
2699,0,0,0,0,0,0,0,1,97,0,0
2700,0,0,0,0,0,0,0,0,98,0,0
